## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-16-18-57-29 +0000,nyt,Senators Investigate Role of A.I. Data Centers...,https://www.nytimes.com/2025/12/16/business/en...
1,2025-12-16-18-52-51 +0000,nypost,Drill rapper Kay Flock couldn’t be gang boss b...,https://nypost.com/2025/12/16/us-news/drill-ra...
2,2025-12-16-18-50-00 +0000,wsj,Comex Gold Settles 0.05% Lower at $4304.50,https://www.wsj.com/finance/commodities-future...
3,2025-12-16-18-47-27 +0000,nyt,M.I.T. Professor Is Fatally Shot in His Home,https://www.nytimes.com/2025/12/16/us/mit-prof...
4,2025-12-16-18-47-00 +0000,wsj,Congressional GOP leaders appeared to close th...,https://www.wsj.com/politics/policy/house-gop-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2379/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
138,trump,43
142,bondi,35
41,reiner,26
587,beach,24
39,rob,20


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
322,2025-12-15-21-15-00 +0000,wsj,Democrats and some Republicans say President T...,https://www.wsj.com/politics/policy/trump-rob-...,132
330,2025-12-15-20-58-52 +0000,latimes,Chabria: Trump's callous political attack on R...,https://www.latimes.com/california/story/2025-...,120
16,2025-12-16-18-29-49 +0000,nyt,"Susie Wiles, Trump’s Chief of Staff, Criticize...",https://www.nytimes.com/2025/12/16/us/politics...,111
198,2025-12-16-09-49-57 +0000,bbc,Trump repeats criticism of killed Hollywood di...,https://www.bbc.com/news/articles/c7413d9xvkeo...,106
241,2025-12-16-02-06-41 +0000,nyt,Trump’s Post Attacking Rob Reiner After Stabbi...,https://www.nytimes.com/2025/12/15/us/politics...,103


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
322,132,2025-12-15-21-15-00 +0000,wsj,Democrats and some Republicans say President T...,https://www.wsj.com/politics/policy/trump-rob-...
153,101,2025-12-16-12-21-54 +0000,nypost,Heroic couple died in each other’s arms after ...,https://nypost.com/2025/12/16/world-news/heroi...
290,69,2025-12-15-22-51-00 +0000,wsj,Police Ask Public to Help Identify Potential B...,https://www.wsj.com/us-news/brown-university-s...
141,56,2025-12-16-13-33-47 +0000,nypost,White House chief of staff Susie Wiles: AG Pam...,https://nypost.com/2025/12/16/us-news/ag-pam-b...
345,56,2025-12-15-20-13-27 +0000,nypost,US close to strongest ever Ukraine security gu...,https://nypost.com/2025/12/15/world-news/us-cl...
329,46,2025-12-15-20-59-29 +0000,nypost,"What is Turtle Island Liberation Front, the ex...",https://nypost.com/2025/12/15/us-news/who-is-t...
303,46,2025-12-15-22-00-24 +0000,bbc,Epstein's UK flights had alleged British abuse...,https://www.bbc.com/news/articles/cy4709kylkxo...
282,37,2025-12-15-23-10-15 +0000,nyt,Ford Will Take $19.5 Billion Hit as It Rolls B...,https://www.nytimes.com/2025/12/15/business/fo...
30,36,2025-12-16-18-11-16 +0000,nypost,Nick Reiner ‘stormed off’ after getting into t...,https://nypost.com/2025/12/16/us-news/nick-rei...
333,35,2025-12-15-20-55-31 +0000,nypost,Holocaust survivors and children among the 15 ...,https://nypost.com/world-news/bondi-beach-shoo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
